The issue was the o2sat factor was not connected to the o2sat var

In [1]:
import src.models.helpers as mh
import src.models.builders as mb
import src.inference.helpers as ih
import src.data.breathe_data as br


import numpy as np
import pandas as pd

In [2]:
df = br.load_meas_from_excel("BR_O2_FEV1_FEF2575_PEF_Nan")

INFO:root:* Checking for same day measurements *


In [3]:
# Remove PEF related rows
df = df.drop(columns=["PEF", "ecPEF (L/s)", "PEF (L/s)", "FEF2575", "ecFEF2575"])
# Remove NaN on FEV1, O2 saturation columns
print(df.shape)
df = df.dropna(subset=["FEV1", "O2 Saturation"])
print(df.shape)
df.head()

(60785, 13)
(45489, 13)


,ID,Date Recorded,FEV1,O2 Saturation,ecFEV1,Sex,Height,Age,Predicted FEV1,Healthy O2 Saturation,ecFEV1 % Predicted,FEV1 % Predicted,O2 Saturation % Healthy
0,101,2019-01-25,1.31,97.0,1.31,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,99.845492
1,101,2019-01-26,1.31,98.0,1.31,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,100.874827
2,101,2019-01-27,1.31,96.0,1.31,Male,173.0,53,3.610061,97.150104,36.287474,36.287474,98.816157
3,101,2019-01-28,1.30,96.0,1.31,Male,173.0,53,3.610061,97.150104,36.287474,36.010470,98.816157
4,101,2019-01-29,1.28,98.0,1.30,Male,173.0,53,3.610061,97.150104,36.010470,35.456463,100.874827


In [6]:
_, inf_alg, HFEV1, ecFEV1, AR, HO2Sat, _, IA, UO2Sat, O2Sat = (
    mb.o2sat_fev1_point_in_time_model_shared_healthy_vars_light(
        170, 30, "Male"
    )
)
res = ih.infer_on_factor_graph(
    inf_alg,
    [O2Sat, UO2Sat],
    [[ecFEV1, 2]],
)
print(res[UO2Sat.name].values)
res[O2Sat.name].values

[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 5.36601532e-32 3.83781691e-16 8.98099819e-05
 1.55818874e-02 6.51944929e-02 6.89771293e-02 6.89771292e-02
 6.89771293e-02 6.89771292e-02 6.89771292e-02 6.89771292e-02
 6.89771293e-02 6.89771292e-02 6.89771293e-02 6.89771293e-02
 6.89771292e-02 6.89751233e-02 6.28497446e-02 2.84242480e-02
 1.36272146e-04]


array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       7.96364002e-34, 5.63827121e-18, 1.23489310e-06, 2.52926236e-04,
       8.46054422e-03, 4.04306630e-02, 6.65746128e-02, 6.86503901e-02,
       6.89311307e-02, 6.88134826e-02, 6.87810489e-02, 6.91011065e-02,
       6.93430429e-02, 6.91742114e-02, 6.88614554e-02, 6.88029776e-02,
       6.88186709e-02, 6.89118793e-02, 6.56709522e-02, 4.49765688e-02,
       1.45685333e-02, 8.74568339e-04])

In [10]:
O2Sat.card

26

In [27]:
from plotly.subplots import make_subplots
import src.models.helpers as mh

def plot_2d_cpt(cpt, cVar, pVar, height=2500, vspace=0.003, invert=False):
    """
    CPT represents P(cVar | pVar)
    The probability distribution of the child variable conditionned on the parent variable
    """
    if invert:
        cpt = cpt.T
        vartmp = cVar
        cVar = pVar
        pVar = vartmp

    plot_bins = range(pVar.card)
    title = f"CPT - P({cVar.get_abbr()}|{pVar.get_abbr()})"

    fig = make_subplots(rows=len(plot_bins), cols=1, shared_xaxes=True, vertical_spacing=vspace)

    for i, idx in enumerate(plot_bins):
        p = cpt[:, idx]
        ih.plot_histogram(fig, cVar, p, cVar.a, cVar.b, i+1, 1, colour='#0072b2', annot=False)
        fig.update_yaxes(title_text=f"{pVar.get_abbr()}={pVar.midbins[idx]:2g}", row=i + 1, col=1, range=[0, 0.6])

    fig.update_layout(
        title=title,
        width=500,
        height=height,
        showlegend=False,
        font=dict(size=8),
    )
    fig.show()
    # fig.write_image(
    #     dh.get_path_to_main()
    #     + f"PlotsBreathe/AR_modelling/{title}.pdf"
    # )

plot_2d_cpt(O2Sat.cpt, O2Sat, UO2Sat, height=2000, invert=True)

In [28]:
plot_2d_cpt(O2Sat.cpt, O2Sat, UO2Sat, height=2000, invert=False)

In [34]:
uo2sat_val = res[UO2Sat.name].values
uo2sat_val.shape

(25,)

In [33]:
O2Sat.cpt.shape

(26, 25)

In [35]:
np.matmul(O2Sat.cpt, uo2sat_val)

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       7.96364002e-34, 5.63827121e-18, 1.23489310e-06, 2.52926236e-04,
       8.46054422e-03, 4.04306630e-02, 6.65746128e-02, 6.86503901e-02,
       6.89311307e-02, 6.88134826e-02, 6.87810489e-02, 6.91011065e-02,
       6.93430429e-02, 6.91742114e-02, 6.88614554e-02, 6.88029776e-02,
       6.88186709e-02, 6.89118793e-02, 6.56709522e-02, 4.49765688e-02,
       1.45685333e-02, 8.74568339e-04])

In [5]:
import plotly.graph_objects as go
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.003)
ih.plot_histogram(fig, UO2Sat, uo2sat_val, UO2Sat.a, UO2Sat.b, 1, 1, colour='#0072b2', annot=False)
# Reduce height
fig.update_layout(height=300)
fig.show()

fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.003)
ih.plot_histogram(fig, O2Sat, np.matmul(O2Sat.cpt, uo2sat_val), O2Sat.a, O2Sat.b, 1, 1, colour='#0072b2', annot=False)
# Reduce height
fig.update_layout(height=300)
fig.show()

NameError: name 'make_subplots' is not defined

In [43]:
O2Sat.midbins

array([ 50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,  66.,  68.,  70.,
        72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,  88.,  90.,  92.,
        94.,  96.,  98., 100.])